# Senata Elections, 1976-2020
This Notebook computes the tables required for the Senate Elections example dashboard.

Step 1: Import the libraries

In [ ]:
import pandas as pd
from galyleo.galyleo_table import GalyleoTable
from galyleo.galyleo_jupyterlab_client import GalyleoClient

Use Pandas to read the libraries, and view the result

In [2]:
elections = pd.read_csv('1976-2020-senate.csv')
elections

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,candidate,party_detailed,writein,mode,candidatevotes,totalvotes,unofficial,version,party_simplified
0,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,SAM STEIGER,REPUBLICAN,False,total,321236,741210,False,20210114,REPUBLICAN
1,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,WM. MATHEWS FEIGHAN,INDEPENDENT,False,total,1565,741210,False,20210114,OTHER
2,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,DENNIS DECONCINI,DEMOCRAT,False,total,400334,741210,False,20210114,DEMOCRAT
3,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,ALLAN NORWITZ,LIBERTARIAN,False,total,7310,741210,False,20210114,LIBERTARIAN
4,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,BOB FIELD,INDEPENDENT,False,total,10765,741210,False,20210114,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3624,2020,WYOMING,WY,56,83,68,US SENATE,statewide,gen,False,UNDER VOTES,NaN,False,total,6401,278503,False,20210114,OTHER
3625,2021,GEORGIA,GA,13,58,44,US SENATE,statewide,runoff,True,KELLY LOEFFLER,REPUBLICAN,False,total,2194848,4483294,True,20210114,REPUBLICAN
3626,2021,GEORGIA,GA,13,58,44,US SENATE,statewide,runoff,True,RAPHAEL WARNOCK,DEMOCRAT,False,total,2288446,4483294,True,20210114,DEMOCRAT
3627,2021,GEORGIA,GA,13,58,44,US SENATE,statewide,runoff,False,DAVID A. PERDUE,REPUBLICAN,False,total,2213979,4483241,True,20210114,REPUBLICAN


Cut out irrelevant columns, compute the percentage, and trim out candidates who got < 1% of the vote

In [23]:
elections_trimmed = elections[['year', 'state', 'special', 'candidate', 'party_simplified', 'candidatevotes', 'totalvotes']]
elections_two = elections_trimmed.assign(percentage=pd.Series(elections_trimmed['candidatevotes']/elections_trimmed['totalvotes'] * 100))
elections_three = elections_two[elections_two['percentage'] > 1]

A convenience method to send a dataframe to a dashboard, with table name name, using GalyleoClient client

In [ ]:
def send_dataframe_to_dashboard_table(dataframe, table_name, client):
    table = GalyleoTable(table)
    table.load_from_dataframe(dataframe)
    client.send_data_to_dashboard(table)

In [ ]:
client = GalyleoClient()
send_dataframe_to_dashboard(elections_three, 'senate_vote', client)

A convenience  method to take a dataframe, add columns for the values in split_column_name, and populate those columns with the appropriate value from values_column_name.  The use case for this in this Notebook is to take the percentage column and turn it into four columns: DEMOCRAT, REPUBLICAN, LIBERTARIAN, AND OTHER.
Parameters: 
  - dataframe: the source dataframe
  - index_columns: the columns to use as indexes -- common across all the daughter tables
  - split_column_name: the column containing the names of the new columns
  - values_column_name: the column containing the values for the new columns

In [88]:
def split_column(dataframe, index_columns, split_column_name, values_column_name):
    keep_columns = index_columns.append(values_column_name)
    table_names = set([v for v in dataframe[split_column_name]])
    tables = []
    for name in table_names:
        table = dataframe[dataframe[split_column_name] == name][keep_columns]
        tables.append(table.rename(columns = {values_column_name: name}).set_index(index_columns))
    result = tables[0]
    for table in tables[1:]:
        result = result.join(table)
    return result.reset_index().fillna(0)

In [89]:
by_party = split_column(elections_three[['year', 'state', 'special', 'party_simplified', 'percentage']], 'party_simplified', 'percentage')

Compute the margin for a row.  This should be +5 to + 10 for a Democrat victory, 0 for OTHER or LIBERTARIAN, and -5 to -10 for a REPUBLICAN victory.  5 is a DEMOCRAT squeaker, 10 a landslide, -5 a REPUBLICAN squeaker, -10 a landslide

In [99]:
import math

def compute_margin(row):
    keys = ['DEMOCRAT', 'REPUBLICAN', 'LIBERTARIAN', 'OTHER']
    parties = [(key, row[key]) for key in keys]
    parties.sort(key=lambda party: party[1], reverse=True)
    if (parties[0][0] in {'LIBERTARIAN', 'OTHER'}): return 0
    margin = min(math.floor((parties[0][1] - parties[1][1])/2), 5)
    return margin + 5 if  parties[0][0] == 'DEMOCRAT' else -(5 + margin)
    
    

In [100]:
by_party['margin'] = by_party.apply(lambda row: compute_margin(row), axis = 1)

Send the vote history table to the dashboard

send_dataframe_to_dashboard(by_party[['year', 'state', 'special', 'REPUBLICAN', 'DEMOCRAT', 'LIBERTARIAN', 'OTHER']], 'history', client)

Send the margin table to the dashboard

In [ ]:
send_dataframe_to_dashboard(by_party[['year', 'state', 'special', 'margin']], 'margin', client)